In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
from math import *
import glob
from IPython.display import display
# from scipy.linalg import block_diag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Сливаем таблицы в один дата фрейм
names = glob.glob(r'/content/drive/MyDrive/Андан Проект/Данные индексов/*.csv',\
                  recursive=True)
ticker_names = [x.replace('.csv', '').split()[-1] for x in names]
merged = pd.read_csv(names[0])
merged = merged.rename(str.lower, axis=1).rename(columns={' ime': 'time',\
                                                 'open': f'open_{ticker_names[0]}',\
                                                 'high': f'high_{ticker_names[0]}',\
                                                 'low': f'low_{ticker_names[0]}',\
                                                 'close': f'close_{ticker_names[0]}',\
                                                 'volume': f'volume_{ticker_names[0]}',\
                                                 })
for i in range(1, len(names)):
    df = pd.read_csv(names[i])
    df = df.rename(str.lower, axis=1).rename(columns={' ime': 'time',\
                                                 'open': f'open_{ticker_names[i]}',\
                                                 'high': f'high_{ticker_names[i]}',\
                                                 'low': f'low_{ticker_names[i]}',\
                                                 'close': f'close_{ticker_names[i]}',\
                                                 'volume': f'volume_{ticker_names[i]}',\
                                                 })
    merged = merged.merge(df, on='time')
    df_indexes = merged

In [ ]:
df_indexes['time'] = pd.to_datetime(df_indexes['time'])

In [ ]:
# Добавляем новый признак логарифмическая доходность за минуту
for ticker_name in ticker_names:
    df_indexes[f'log_return_{ticker_name}'] = np.log(df_indexes[f'open_{ticker_name}'].shift(1) / \
    df_indexes[f'open_{ticker_name}'])

In [ ]:
# Провальная попытка сделать матрично
# n, d = 390, 13
# blocks = [np.ones((n, n)) * 1/n] * d
# day_mean_matrix = block_diag(*blocks)[1:, :]
# for ticker_name in ticker_names:
#     np_mean_day_log_return = day_mean_matrix @ df_indexes[f'log_return_{ticker_name}'].to_numpy()
#     print(np_mean_day_log_return)
#     df_indexes[f'mean_day_log_return_{ticker_name}'] = pd.DataFrame(np_mean_day_log_return)

In [ ]:
df_indexes['open_QQQ'].corr(df_indexes['open_SPX'])

0.9831245751946432

In [ ]:
df_indexes['open_QQQ'].corr(df_indexes['open_SPY'])

0.985977407698796

In [ ]:
df_indexes['open_SPX'].corr(df_indexes['open_SPY'])

0.9995977119568936

In [ ]:
# Добавляем новый признак минутная волатильность
df_indexes['date'] = df_indexes['time'].apply(lambda x: x.floor('d'))
for ticker_name in ticker_names:
    df_std = df_indexes.groupby(pd.Grouper(key='time', freq='D'))\
     [f'log_return_{ticker_name}'].std()
    df_std = df_std.reset_index()
    df_std.rename(columns={'time': 'date', f'log_return_{ticker_name}':\
                           f'mean_std_log_return_{ticker_name}'}, inplace=True)
    df_indexes = df_indexes.merge(df_std, on='date', how='left')
df_indexes.drop(columns='date', inplace=True)

In [ ]:
df_indexes.head()

,time,open_QQQ,high_QQQ,low_QQQ,close_QQQ,volume_QQQ,open_SPY,high_SPY,low_SPY,close_SPY,...,high_SPX,low_SPX,close_SPX,volume_SPX,log_return_QQQ,log_return_SPY,log_return_SPX,mean_std_log_return_QQQ,mean_std_log_return_SPY,mean_std_log_return_SPX
0,2025-02-03 09:30:00,513.47,513.54,512.58,513.28,85501,592.67,592.92,591.92,592.86,...,5948.14,5937.49,5943.86,156821116,NaN,NaN,NaN,0.000704,0.00058,0.000558
1,2025-02-03 09:31:00,513.22,513.88,512.98,513.81,37627,592.91,593.36,592.72,593.32,...,5950.94,5944.50,5950.84,16252045,0.000487,-0.000405,0.000605,0.000704,0.00058,0.000558
2,2025-02-03 09:32:00,513.90,514.39,513.65,514.02,34127,593.34,594.06,593.34,593.87,...,5958.20,5950.68,5957.25,16231682,-0.001324,-0.000725,-0.001032,0.000704,0.00058,0.000558
3,2025-02-03 09:33:00,514.03,514.25,513.52,513.62,31471,593.89,594.13,593.43,593.61,...,5958.67,5952.60,5953.87,14275633,-0.000253,-0.000927,-0.001172,0.000704,0.00058,0.000558
4,2025-02-03 09:34:00,513.55,513.89,513.27,513.66,31860,593.61,593.85,593.18,593.54,...,5956.21,5951.07,5953.46,14437487,0.000934,0.000472,0.000651,0.000704,0.00058,0.000558
